<a href="https://colab.research.google.com/github/casualcomputer/llm_google_colab/blob/main/llm_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install dependencies
!pip install --upgrade pip --q
!pip install langchain google-generativeai torch torchvision transformers datasets langchain_community --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [9]:
# Imports & Authentication
import os
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.agents import initialize_agent, Tool, AgentType
from datasets import load_dataset
import re
import pandas as pd
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY') # Use the name you gave your secret

if api_key:
    # Use the api_key
    print("API key loaded successfully from Colab Secrets.")
    # print(f"Your API key is: {api_key}") # Be careful printing the key itself
else:
    print("API key 'GOOGLE_API_KEY' not found in Colab Secrets.")

API key loaded successfully from Colab Secrets.


In [10]:
!pip install langchain_google_genai --quiet
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest", # You can also try "gemini-pro"
            temperature=0,
            google_api_key=api_key
        )
print("LLM Initialized successfully.")

print("\nSending a test prompt to the LLM...")
print(type(llm))
print(llm.invoke("Write me a ballad about LangChain").content)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
LLM Initialized successfully.

Sending a test prompt to the LLM...
<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>
(Verse 1)
The data streams, a raging flood,
A chaos vast, misunderstood.
To tame its power, find its core,
A programmer sought, and something more.
He heard a whisper, on the breeze,
Of LangChain's promise, meant to please.
A framework new, a shining light,
To bring the darkness into sight.

(Verse 2)
With LLMs vast, and prompts so keen,
He built a chain, a wondrous scene.
From prompt engineering's subtle art,
He crafted queries, played his part.
The chains he forged, both long and deep,
Where memories slept, and secrets keep.
He linked th

# Agent

In [13]:
!pip install crewai crewai_tools --quiet

from crewai import Agent, Task, Crew, LLM

# 1) Define a simple search tool (or replace with your real search API)
def search_tool(query: str) -> str:
    # Right now it’s a stub. In production you might call Wikipedia, SerpAPI, etc.
    return "Ottawa (capital of Canada), population ~1 million"

# 2) Instantiate your CrewAI Agent with the Search tool registered
agent_react = Agent(
    role="Factual Researcher",
    goal="Retrieve factual snippets for user queries",
    backstory="Always think step-by-step and use external tools when needed.",
    verbose=True,            # prints Thought/Action/Observation logs
    allow_delegation=False,
    llm=llm
    #, tools=[
    #     {"name": "Search", "func": search_tool, "description": "Lookup facts online"}
    # ]
)


In [25]:
# 3. Define all your prompt templates from Section 1
ZERO_SHOT_TEMPLATE = "You are a helpful assistant. Answer concisely:\n\n{query}"
FEW_SHOT_TEMPLATE = (
    "You are a helpful assistant. Use the examples below to guide your answer.\n\n"
    "Example 1:\nQ: Who wrote 'Pride and Prejudice'?\nA: Jane Austen\n\n"
    "Example 2:\nQ: What is the boiling point of water in Celsius?\nA: 100°C\n\n"
    "Now answer:\nQ: {query}\nA:"
)
COT_TEMPLATE = (
    "You are a reasoning assistant. Think step by step:\n\n"
    "Question: {query}\n"
    "Answer: Let’s think step by step."
)
COT_SC_TEMPLATE = (
    "You are a reasoning assistant. Think carefully, step by step:\n\n"
    "Question: {query}\n"
    "Answer:"
)
REACT_TEMPLATE = (
    "You are a multi-agent reasoning assistant. Follow the ReAct format exactly:\n\n"
    "Thought: I need to gather facts for: {query}\n"
    "Action: Search[\"{query}\"]\n"
    "Observation:\n"
    "Thought: Now I have the necessary information.\n"
    "Answer:"
)
RAG_TEMPLATE = (
    "You have been provided the following context passages:\n\n{retrieved_docs}\n\n"
    "Using only the above context, answer:\n\n{query}"
)
CHAIN_STEP1 = (
    "Extract key facts needed to answer:\n\n"
    "Question: {query}\n"
    "Facts:"
)
CHAIN_STEP2 = (
    "You have these facts:\n\n{facts}\n\n"
    "Now answer:\n\n"
    "Question: {query}\n"
    "Answer:"
)
META_PROMPT = (
    "You are a prompt engineer. Given the user’s instruction, produce an optimized prompt:\n\n"
    "User instruction: {user_instruction}\n\n"
    "Optimized prompt:"
)
PAL_TEMPLATE = (
    "You are a Python interpreter. Write Python code that solves:\n\n"
    "{query}\n\n"
    "# After writing the code, execute it mentally and provide the result."
)

# 4. Build a dispatcher function to select the right template
def build_prompt(query: str, mode: str, **kwargs) -> str:
    if mode == "zero_shot":
        return ZERO_SHOT_TEMPLATE.format(query=query)
    elif mode == "few_shot":
        return FEW_SHOT_TEMPLATE.format(query=query)
    elif mode == "cot":
        return COT_TEMPLATE.format(query=query)
    elif mode == "cot_sc":
        return COT_SC_TEMPLATE.format(query=query)
    elif mode == "react":
        return REACT_TEMPLATE.format(query=query)
    elif mode == "rag":
        return RAG_TEMPLATE.format(retrieved_docs=kwargs.get("retrieved_docs", ""), query=query)
    elif mode == "chain_step1":
        return CHAIN_STEP1.format(query=query)
    elif mode == "chain_step2":
        return CHAIN_STEP2.format(query=query, facts=kwargs.get("facts", ""))
    elif mode == "meta":
        return META_PROMPT.format(user_instruction=query)
    elif mode == "pal":
        return PAL_TEMPLATE.format(query=query)
    else:
        raise ValueError(f"Unknown mode: {mode}")

# # 5. Helper to call either the Agent (for ReAct) or bare LLM
# def call_with_mode(query: str, mode: str, **kwargs) -> str:
#     prompt = build_prompt(query, mode, **kwargs)
#     if mode == "react":
#         response = agent_react.llm.invoke(prompt)
#         return response.content.strip()
#     else:
#         response = llm.invoke(prompt)
#         return response.content.strip()

# # 5. Helper to call either the Agent (for ReAct) or bare LLM
# def call_with_mode(query: str, mode: str, **kwargs) -> str:
#     prompt = build_prompt(query, mode, **kwargs)

#     if mode == "react":
#         # “Call” the agent’s LLM so that Search[…] actually invokes your registered tool
#         ai_msg = agent_react.llm(prompt)
#         return ai_msg.content.strip()
#     else:
#         # “Call” the bare LLM for all other modes
#         ai_msg = llm(prompt)
#         return ai_msg.content.strip()
# Solution 2
# # -------  helper that always extracts text  -------
# def _to_text(resp):
#     """
#     CrewAI/Gemini may return either:
#       • a plain str  (newer versions)
#       • an AIMessage (older LangChain chat models)
#     This helper normalises both to a clean string.
#     """
#     from langchain_core.messages import BaseMessage  # only used for isinstance check
#     if isinstance(resp, str):
#         return resp
#     elif isinstance(resp, BaseMessage):      # covers AIMessage, HumanMessage, etc.
#         return resp.content
#     else:
#         # Fallback: best-effort string conversion
#         return str(resp)

from langchain_core.messages import BaseMessage   # for the isinstance check

def _to_text(resp) -> str:
    """Normalise CrewAI responses to a clean string."""
    if isinstance(resp, str):
        return resp
    if isinstance(resp, BaseMessage):
        return resp.content
    return str(resp)          # fallback

def smart_llm_call(model, prompt):
    """
    Call the LLM using whichever entry-point the installed
    CrewAI / LangChain version supports.
    """
    for m in ("invoke", "predict", "__call__"):
        if hasattr(model, m):
            resp = getattr(model, m)(prompt)     # call the method
            return _to_text(resp)
    raise AttributeError("LLM exposes no invoke/predict/__call__")

# ------------------ MAIN DISPATCHER ------------------
def call_with_mode(query: str, mode: str, **kwargs) -> str:
    prompt = build_prompt(query, mode, **kwargs)
    if mode == "react":          # needs the Search tool
        return smart_llm_call(agent_react.llm, prompt).strip()
    else:                        # every other prompting style
        return smart_llm_call(llm, prompt).strip()


# # -------  ONE call-site for every prompting mode  -------
# def call_with_mode(query: str, mode: str, **kwargs) -> str:
#     prompt = build_prompt(query, mode, **kwargs)

#     if mode == "react":
#         # ReAct needs the agent so the Search tool can fire
#         resp = agent_react.llm.invoke(prompt)          # ✅ modern, no deprecation warning
#     else:
#         resp = llm.invoke(prompt)                      # ✅ modern, no deprecation warning

#     return _to_text(resp).strip()

# 6. Example Calls
query_example = "What is the capital of Canada and its population?"

# 6A. Zero-Shot
res_zero = call_with_mode(query_example, mode="zero_shot")
print("Zero-Shot:", res_zero)  # :contentReference[oaicite:24]{index=24}
print("...................")

# 6B. Few-Shot
res_few = call_with_mode(query_example, mode="few_shot")
print("Few-Shot:", res_few)    # :contentReference[oaicite:25]{index=25}
print("...................")

# 6C. Chain-of-Thought (CoT)
res_cot = call_with_mode(query_example, mode="cot")
print("CoT:", res_cot)         # :contentReference[oaicite:26]{index=26}
print("...................")

# 6D. Self-Consistency (CoT-SC) (voting over 3 runs)
cot_sc_chains = [call_with_mode(query_example, mode="cot_sc") for _ in range(3)]
finals = [chain.split("\n")[-1].strip() for chain in cot_sc_chains if "Answer:" in chain]
from collections import Counter
res_cot_sc = Counter(finals).most_common(1)[0][0] if finals else ""
print("CoT-SC Vote:", res_cot_sc)  # :contentReference[oaicite:27]{index=27}
print("...................")

# # 6E. ReAct (tool call)
# res_react = call_with_mode(query_example, mode="react")
# print("ReAct:", res_react)     # :contentReference[oaicite:28]{index=28}

# 6G. Prompt Chaining (Two-Step)
facts = call_with_mode(query_example, mode="chain_step1")
print("Chained Facts:", facts)  # :contentReference[oaicite:30]{index=30}
res_chain_final = call_with_mode(query_example, mode="chain_step2", facts=facts)
print("Chained Final:", res_chain_final)  # :contentReference[oaicite:31]{index=31}
print("...................")

# 6H. Meta Prompting
opt_prompt = call_with_mode("Optimize a prompt for: " + query_example, mode="meta")
print("Meta Optimized Prompt:", opt_prompt)  # :contentReference[oaicite:32]{index=32}
print("...................")

# 6I. Program-Aided Language Model (PAL)
res_pal = call_with_mode(query_example, mode="pal")
print("PAL:", res_pal)         #
print("...................")

# # 6F. Retrieval-Augmented Generation (RAG)
# # Assume retrieve_docs is implemented elsewhere
# def retrieve_docs(q: str, k: int = 3) -> str:
#     # E.g., use a FAISS vector store or Wikipedia API
#     return "Ottawa is the capital of Canada. The population is ~1 million."

# docs = retrieve_docs(query_example, k=3)
# res_rag = call_with_mode(query_example, mode="rag", retrieved_docs=docs)
# print("RAG:", res_rag)         # :contentReference[oaicite:29]{index=29}

Zero-Shot: Ottawa; approximately 1 million.
...................
Few-Shot: The capital of Canada is Ottawa.  Its population is approximately 1 million.
...................
CoT: Let’s think step by step.

Step 1: Identify the question. The question asks for two pieces of information: the capital city of Canada and its population.

Step 2: Determine the capital city.  The capital city of Canada is Ottawa.

Step 3: Determine the population.  The population of Ottawa is not a fixed number as it changes constantly.  To answer accurately, we need to specify a time frame (e.g., the population as of a specific year or a recent estimate).  A quick online search would provide the most up-to-date information.

Step 4:  Combine the information.  Once a reliable population figure for Ottawa is found (e.g., from Statistics Canada or a reputable news source), the complete answer can be formulated as: "The capital of Canada is Ottawa.  [Insert population figure here] is its approximate population as of

In [28]:
query = "What is the capital of Australia and its population?"
for mode in ["zero_shot", "few_shot", "cot", "cot_sc"]:
    print(f"{mode:<8} →", call_with_mode(query, mode))

zero_shot → Canberra; approximately 430,000
few_shot → A: The capital of Australia is Canberra.  Its population is approximately 430,000.
cot      → Let’s think step by step.

Step 1: Identify the country in question. The question asks about the capital of Australia.

Step 2: Determine the capital city. The capital city of Australia is Canberra.

Step 3: Find the population of the capital city.  A quick online search reveals that the population of Canberra is approximately 430,000 (this number fluctuates and depends on the source and year).

Step 4: Combine the findings.  Therefore, the capital of Australia is Canberra, and its population is approximately 430,000.
cot_sc   → Step 1: Identify the question. The question asks for two pieces of information: the capital city of Australia and its population.

Step 2: Determine the capital city. The capital city of Australia is Canberra.

Step 3: Find the population.  A quick online search reveals that the population of Canberra fluctuates, b